In [1]:
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # 👈 Ignorar imágenes corruptas

import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [ ]:
base_dir = r"C:\Users\hamga\Documents\repo\ProcesamientoYClasificacionDeDatosProyecto\Bone_Fracture_Binary_Classification"
batch_size = 32
num_epochs = 1
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

🖥️ Usando dispositivo: cpu


In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = datasets.ImageFolder(os.path.join(base_dir, "train"), transform=train_transforms)
val_dataset   = datasets.ImageFolder(os.path.join(base_dir, "val"), transform=val_test_transforms)
test_dataset  = datasets.ImageFolder(os.path.join(base_dir, "test"), transform=val_test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Clases detectadas: {train_dataset.classes}")
print(f"Imágenes - Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


✅ Clases detectadas: ['fractured', 'not fractured']
📊 Imágenes - Train: 9246, Val: 829, Test: 506


In [5]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 clases: fractured / not fractured
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)


c:\Users\hamga\Documents\repo\ProcesamientoYClasificacionDeDatosProyecto\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hamga\Documents\repo\ProcesamientoYClasificacionDeDatosProyecto\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")

    # ==========================
    # VALIDACIÓN
    # ==========================
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Validación Accuracy: {val_acc:.2f}%")

🟡 Epoch 1/1 - Loss: 0.0710


c:\Users\hamga\Documents\repo\ProcesamientoYClasificacionDeDatosProyecto\venv\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Validación Accuracy: 98.67%


In [ ]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = 100 * correct / total
print(f"Accuracy en Test: {test_acc:.2f}%")

🎯 Accuracy en Test: 99.01%


In [ ]:
torch.save(model.state_dict(), "modelo_fractura_resnet50.pth")

💾 Modelo guardado como 'modelo_fractura_resnet50.pth'
